In [1]:
import observational_fear.load as load
from observational_fear.events import get_freeze_starts
from neurobox.wide_transforms import resample
from neurobox.compose import Pipeline
from observational_fear.nbox_transforms import (
    align_to_data_by,
    exclude_short_trials,
)
import pandas as pd
from observational_fear.stats import auc
from pathlib import Path
from observational_fear.events import get_freeze_stops
import numpy as np


DATA_DIR = Path(r"F:\OFL\one-p experiment\data")
cache_dir = DATA_DIR / "cached"
FIG_DIR = Path(r"C:\Users\roryl\repos\observational-fear\figs")

In [2]:
def get_traces(data_dir, session):
    df_cells = load.load_cells(data_dir)
    df_cells["new_id"] = df_cells["new_id"].astype("str")
    df_traces = load.load_traces(data_dir, session=session)
    df_traces = resample(df_traces.set_index("time").copy(), "100ms")
    df_traces = df_traces.reset_index().melt(id_vars=["time"]).merge(df_cells)
    return df_traces


def get_freezes(data_dir, session, start_stop="start", role="obs"):
    if session == "day2":
        freeze_col = f"was_freezing_{role}"
        df_freeze = load.load_freeze(data_dir, session=session)
    else:
        freeze_col = "was_freezing"
        df_freeze = load.load_behaviour(data_dir, session=session)
    if start_stop == "start":
        freeze_func = get_freeze_starts
    elif start_stop == "stop":
        freeze_func = get_freeze_stops
    else:
        raise ValueError("start stop not specified")
    df_events = freeze_func(df_freeze, freeze_col=freeze_col, mouse_col="mouse_name")
    df_events = df_events.rename(columns={"mouse_name": "mouse"})
    return df_events

def get_dummy_freezes(data_dir, session, start_stop="start", role="obs", max_time = 2150):
    return (
        get_freezes(data_dir, session, start_stop=start_stop, role=role)
        .groupby("mouse", as_index=False)
        .apply(lambda x: pd.Series({f"freeze_{start_stop}": np.round(np.random.rand(len(x)) * max_time, 2)}))
        .explode(f"freeze_{start_stop}")
        .assign(**{f"freeze_{start_stop}": lambda x: x[f"freeze_{start_stop}"].astype("float64")})
        .sort_values(["mouse", f"freeze_{start_stop}"])
        .reset_index(drop=True)
    )

def align_traces_to_freeze(
    session, start_stop, time_before=5, time_after=5, role="obs", dummy=False,
):
    if not dummy:
        freeze_getter = get_freezes
    else:
        freeze_getter = get_dummy_freezes
    df_traces = get_traces(data_dir=DATA_DIR, session=session)
    df_events = freeze_getter(DATA_DIR, session=session, role=role, start_stop=start_stop)
    aligned = align_to_data_by(
        df_data=df_traces,
        df_data_time_col="time",
        df_data_cell_col="new_id",
        df_data_value_col="value",
        df_data_group_col="mouse",
        df_events=df_events,
        df_events_timestamp_col=f"freeze_{start_stop}",
        df_events_group_colname="mouse",
        precision=1,
        time_before_event=time_before,
        time_after_event=time_after,
    )
    # aligned = exclude_short_trials(
    #     df=aligned,
    #     trial_col="event",
    #     min_bins_pre_event=40,
    #     min_bins_post_event=40,
    #     cell_col="new_id",
    # )
    return aligned

def save_cache(df: pd.DataFrame, fname: str, cache_dir: Path):
    df.to_parquet(cache_dir / f"{fname}.parquet.gzip", compression="gzip", index=False)
    

In [8]:
day2_even = align_traces_to_freeze("day2", start_stop="start", time_before=5, time_after=5, role="obs")
save_cache(day2_even, fname="day2_freeze_start_even", cache_dir=cache_dir)

day2_even = align_traces_to_freeze("day2", start_stop="stop", time_before=5, time_after=5, role="obs")
save_cache(day2_even, fname="day2_freeze_stop_even", cache_dir=cache_dir)

mouse            object
freeze_start    float64
dtype: object
mouse           object
freeze_stop    float64
dtype: object


In [9]:
day4_even = align_traces_to_freeze("day4", start_stop="start", time_before=5, time_after=5, role="obs")
save_cache(day4_even, fname="day4_freeze_start_even", cache_dir=cache_dir)

day4_even = align_traces_to_freeze("day4", start_stop="stop", time_before=5, time_after=5, role="obs")
save_cache(day4_even, fname="day4_freeze_stop_even", cache_dir=cache_dir)

mouse            object
freeze_start    float64
dtype: object
mouse           object
freeze_stop    float64
dtype: object


In [10]:
day2_even = align_traces_to_freeze("day2", start_stop="start", time_before=5, time_after=5, role="obs", dummy=True)
save_cache(day2_even, fname="day2_freeze_start_even_dummy", cache_dir=cache_dir)

day2_even = align_traces_to_freeze("day2", start_stop="stop", time_before=5, time_after=5, role="obs", dummy=True)
save_cache(day2_even, fname="day2_freeze_stop_even_dummy", cache_dir=cache_dir)

mouse            object
freeze_start    float64
dtype: object
mouse           object
freeze_stop    float64
dtype: object


In [11]:
day2_even = align_traces_to_freeze("day4", start_stop="start", time_before=5, time_after=5, role="obs", dummy=True)
save_cache(day2_even, fname="day4_freeze_start_even_dummy", cache_dir=cache_dir)

day2_even = align_traces_to_freeze("day4", start_stop="stop", time_before=5, time_after=5, role="obs", dummy=True)
save_cache(day2_even, fname="day4_freeze_stop_even_dummy", cache_dir=cache_dir)

mouse            object
freeze_start    float64
dtype: object
mouse           object
freeze_stop    float64
dtype: object
